<a href="https://colab.research.google.com/github/salvapineda/notebooks/blob/main/LinearProgramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Programming in Python

This notebook compares different alternatives to solve optimization problems in Python

We solve a standard linear programming problem of the form

```
min  c*x
s.t. A*x <= b
     x >= 0
```

## Create random vectors and matrix of linear problem

In [46]:
# Import
import random
# Number of variables and constraints
nvar = 10
ncon = 5
# Random vectors and matrix of linear problem
c = [round(abs(random.gauss(0,1)),2) for i in range(nvar)]
A = [[round(random.gauss(0,1),2) for i in range(nvar)] for j in range(ncon)]
b = [round(random.gauss(0,1),2) for j in range(ncon)]

## Solve linear problem using SCIPY

In [ ]:
# Import
from scipy.optimize import linprog
# Solve problem
res = linprog(c, A_ub=A, b_ub=b)
# Print results
print(res)

## Solve linear problem using PULP

In [ ]:
# Install
!pip install pulp
# Import
import pulp as pu
# Model
m = pu.LpProblem('LinearProblem',pu.LpMinimize)
# Variables
x = pu.LpVariable.dicts('x', list(range(nvar)), lowBound=0, cat="Continuous")
# Objective function
m += sum(c[i]*x[i] for i in range(nvar))
# Constraints
for j in range(ncon):
  m += sum(A[j][i]*x[i] for i in range(nvar)) <= b[j]
# Solve problem
m.solve()
# Princ results
print('Status:', pu.LpStatus[m.status])
print('Optimal value:',pu.value(m.objective))
print('Optimal solution:',[x[i].varValue for i in range(nvar)])

## Solve linear problem using PYOMO

In [ ]:
# Install
!pip install pyomo
# Import
import os
import pyomo.environ as pe
# Model
m = pe.ConcreteModel()
# Sets
m.i = pe.Set(initialize=range(nvar),ordered=True)
m.j = pe.Set(initialize=range(ncon),ordered=True)
# Variables
m.z = pe.Var()
m.x = pe.Var(m.i,within=pe.NonNegativeReals)
# Objective function
def obj_rule(m):
  return sum(c[i]*m.x[i] for i in m.i)
m.obj = pe.Objective(rule=obj_rule)
# Constraints
def con_rule(m,j):
  return sum(A[j][i]*m.x[i] for i in m.i) <= b[j]
m.con = pe.Constraint(m.j,rule=con_rule)
# Solve problem using NEOS server
os.environ['NEOS_EMAIL'] = 'xxx@gmail.com'
res = pe.SolverManagerFactory('neos').solve(m,opt=pe.SolverFactory('cplex'))
print(res['Solver'][0])
# Print results
print('Optimal value:',m.obj())
print('Optimal solution:',[m.x[i].value for i in m.i])